In [1]:
from watsonx import WatsonxAI

proxy = "proxy.us.ibm.com:8080"

wx = WatsonxAI()
wx.connect()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'iam.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


some basic classes

In [2]:
class Agent:
    name = "Agent"
    model = "ibm/granite-13b-chat-v2"
    instructions = ["You are a helpful agent."]
    functions = []
    tool_choice = None
    parallel_tool_calls = True


class Response:
    messages = []
    agent = None
    context_variables = {}


class Result:
    value = ""
    agent = None
    context_variables = {}

In [3]:
def gen_plan(requirement):
    prompt = f"""[INST]
please generate a concise and solid plan to satisfy the requirement provided.
-understand the requirement in detail.
-list out what knowlege you need to satisfy the requirement.
-think logically.
-think step by step.
-you trust the result of the tools more than your memory.
-you trust the knowledge provide more than your memory.
-leverage tool if possible.
-dont guess.
-please breakdown complex task to simple tasks.
-DONT generate result of the steps.
-**DONT** generate **Final Result:**
<<SYS>>
requirement: `{requirement}`
<</SYS>>
[/INST]plan:"""

    return wx.watsonx_gen(prompt,wx.LLAMA_3_70B_INSTRUCT).replace("```","")

In [4]:
def gen_summary(requirement,answer):
    prompt = f"""[INST]
please generate a complete sentence summary base on the answer provided only.
-use simple english.
-dont guess.
<<SYS>>
requirement: `{requirement}`
answer: `{answer}`
<</SYS>>
[/INST]answer summary:"""

    return wx.watsonx_gen(prompt,wx.LLAMA_3_70B_INSTRUCT).replace("```","")

In [5]:
tools = """[
    get_hko_weather()->str,
    query_duckduckgo(query:str)->str,
    search_arxiv(query:str)->str,
    query_wikipedia(title:str)->str
]"""

In [6]:
def gen_code(requirement,plan):
    prompt = f"""[INST]
please generate python code that execute the plan to fulfill the requirement.
-ensure the python generated is executable.
-clean up if any unneccessary characters.
-you can use the tool provided directly without define it.
-understand the plan in detail. dont miss the steps.
-**DONT** use SaaS service which need API KEY.
-**DONT** catch error.
-raise error when hit problem.
-**DONT** throw SystemExit
-**DONT** generate main().
-**DONT** generate explanation.
-generate the code directly.
-generate the code only.
<<SYS>>
requirement: `{requirement}`
plan: `{plan}`
tools: {tools}
<</SYS>>
[/INST]
```python
"""
    
    #-dont define function.

    return wx.watsonx_gen(prompt,wx.LLAMA_3_70B_INSTRUCT).replace("```","")
    # print(code)

In [7]:
def revise_code(code, plan, error):
    prompt = f"""[INST]
revise the code provided.
-review the problem in the code and the error.
-prevent the error from the code.
-you can use the tool provided directly without define it.
-understand the plan in detail. dont miss the steps.
-**DONT** use SaaS service which need API KEY.
-**DONT** catch error.
-raise error when hit problem. 
-**DONT** throw SystemExit
-**DONT** generate main().
-**DONT** generate explanation.
-generate the code directly.
-generate the code only.
<<SYS>>
code: `{code}`
plan: `{plan}`
error: `{error}`
tools: {tools}
<</SYS>>
[/INST]
fixed code:```python"""

    answer = wx.watsonx_gen(prompt,wx.LLAMA_3_70B_INSTRUCT).replace("```","")
    return answer

In [8]:
import io
import sys
import traceback
from IPython.core.interactiveshell import InteractiveShell
def call_tool(module_import, function_call, max_traceback_lines=5):
    # Create StringIO objects to capture standard output and error output
    output_stream = io.StringIO()
    error_stream = io.StringIO()

    # Get the current interactive shell
    shell = InteractiveShell.instance()

    # Redirect stdout and stderr
    old_stdout = sys.stdout
    old_stderr = sys.stderr
    sys.stdout = output_stream
    sys.stderr = error_stream

    output = None
    error = None

    # Using exec to import the module and evaluate the function call
    try:
        local_namespace = {}
        exec(module_import, local_namespace, local_namespace)  # Dynamically import the module
        print(f"|{function_call}|")
        exec(function_call, local_namespace, local_namespace)  # Evaluate the function call
    except ImportError as e:
        error = f"Import Error: {str(e)}"
    except Exception as e:
        # Capture the traceback for more detailed error information
        full_traceback = traceback.format_exc().strip()
        # Limit the traceback to the last 'max_traceback_lines' lines
        limited_traceback = "\n".join(full_traceback.splitlines()[-max_traceback_lines:])
        error = f"Unexpected Error: {str(e)}\n{limited_traceback}"
    finally:
        # Reset stdout to its original state
        sys.stdout = old_stdout
        sys.stderr = old_stderr

    # Get the output from the output_stream
    output = output_stream.getvalue().strip()
    error_output = error_stream.getvalue().strip()

    # Check the output or error
    if error:
        return f"Output: {output}", f"Error: {error}"
    elif error_output:
        return f"Output: {output}", f"Error Output: {error_output}"
    else:
        return f"Output: {output}", None

In [9]:
def eval_output(question, stdout):
    prompt = f"""[INST]classify if stdout include error.
-show sucess or fail only, but nothing else.
-show success if include no error. and end the generation.
-show fail if include error. and end the generation.
<<SYS>>
question: `{question}`
stdout: `{stdout}`
<</SYS>>
[/INST]success or fail:"""
    success = wx.watsonx_gen(prompt,wx.LLAMA_3_70B_INSTRUCT)
    # print(success)
    return success

In [10]:
def agentic_prompt(requirement, max_turns=10,debug=False):
    # Generate initial action and code
    plan = gen_plan(requirement)
    if debug:
        print(f"#1: {plan}")
    code = gen_code(requirement, plan)
    if debug:
        print(f"#2: <|{code}|>")

    output = ""

    # Loop until the review output is empty
    for turn in range(max_turns):
        print(f"turn #{turn+1} ---")
        # output = exec_with_output(code)
        output, error = call_tool("",code)
        if debug:
            print(f"#3: {output}")
        
        if error is None:
            break
        
        if debug:
            print(f"error {error}")
        # review = eval_output(requirement, output)
        # if debug:
        #     print(f"#4: {review}")
        
        # if "success" in review:  # Exit if review output is empty
        #     # print(f"#final result: {output}")
        #     break
        
        revised = revise_code(code, plan, error)
        if debug:
            print(f"#5: {revised}")

        # Update code for the next iteration
        code = revised

    summary = gen_summary(requirement,output)
    return summary

In [ ]:
output = agentic_prompt("how is weather today in Hong Kong, base on HKO",debug=True)
print(output)

In [ ]:
output = agentic_prompt("what is the population in hong kong",debug=True)
print(output)

In [ ]:
output = agentic_prompt("how many prime number with in 10000")
print(output)

In [ ]:
output = agentic_prompt("how many R in straberry, not regarding to upper lower case")
print(output)

In [ ]:
output = agentic_prompt("in math, compare 9.11 with 9.8")
print(output)

In [ ]:
output = agentic_prompt("tell me Hong Kong time")
print(output)

In [ ]:
active_agent = None
context_variables = []
history = [] 
init_len = 0

max_turns = 10
model_id = wx.GRANITE_20B_CODE_INSTRUCT
tools = []
messages = []
active_agent = Agent(model_id,"do math")
while len(history) - init_len < max_turns:
    message = ""
    partial_response = watsonx.watsonx_gen(message,history)
    history.append(message)
    history.append(partial_response.message)
    context_variables.update(partial_response.ontext_variable)
    active_agent = partial_response.agent

In [ ]:
def clock():
    print("12pm")
    return "12pm"

def calculate(expression):
    return eval(expression)

prompt = f"""[INST]
tell me 3434 * 21 / 32
-for any math question, dont base on memory. ride on tool.
-tell what you confidence to answer, dont guess.
-generate the python code to use tool in need.
-only use the tool provided.
-dont repeat.
-generate the answer or code only.
<<SYS>>
tools: [
    clock()->str #get time,
    abc(code:str)->str #get stock price, 
    calculator(expression:str)->str,
    browser(url:str)->str
    ]
<</SYS>>
[/INST]answer:
"""

answer = wx.watsonx_gen(prompt,wx.LLAMA_3_70B_INSTRUCT)
print(answer)
out = call_tool("",answer.strip())
print(out)

In [ ]:
print(3434 * 21 / 32)

In [ ]:
def clock():
    print("12pm")
    return "12pm"

def ask_granite(question):
    prompt = f"""<|system|>
-tell what you confidence to answer only, dont guess.
-select the right tool.
-dont base on memory. call the right tool in contenxt.
[TOOLS]
tools: [
    clock()->str #get time,
    abc(code:str)->str #get stock price, 
    calculator(expression:str)->str # do math,
    browser(url:str)->str #find informatino
    ]
[/TOOLS]
-generate the python code to use tool in need.
-only use the tool provided.
-dont repeat.
-generate the answer or code only.
-dont generate result.
-dont generate Calculation steps.
-generate the code only.
<|user|>
{question}
<|assistant|>
```python"""

    answer = wx.watsonx_gen(prompt,wx.GRANITE_13B_CHAT_V2)
    return answer

answer = ask_granite("tell me the content of cnn.com today")
print(answer)

answer = ask_granite("tell me 320*232 / 2322")
print(answer)

answer = ask_granite("tell me about paris")
print(answer)